In [ ]:
# Instalación unificada de dependencias
import sys

!{sys.executable} -m pip install \
    langchain==0.1.13 \
    langchain-community==0.0.29 \
    chromadb==0.4.14 \
    sentence-transformers \
    transformers \
    torch \
    pandas \
    wikipedia-api

print("\n--- ¡Dependencias listas! ---")


--- ¡Dependencias listas! ---


In [1]:
# Definición de Rutas

import os

# Estas carpetas se crearán en el entorno temporal de Colab
DATA_DIR = "data"
OUTPUT_DIR = "outputs"
DB_DIR = "chroma_db_store" # Para la base de datos vectorial

# Creamos las carpetas en el entorno local de Colab
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

# Definimos el nombre del artículo de Wikipedia
PAGE_TITLE = "Federated_learning"
CSV_PATH = os.path.join(DATA_DIR, "wiki_corpus.csv")

print(f"Ruta de Datos: {DATA_DIR}")
print(f"Ruta de Salidas: {OUTPUT_DIR}")
print(f"Ruta de DB Vectorial: {DB_DIR}")

Ruta de Datos: data
Ruta de Salidas: outputs
Ruta de DB Vectorial: chroma_db_store


In [2]:
#  Creación del Dataset (Extracción y Chunking)

import wikipediaapi
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

print(f"Creación del Dataset")

# EXTRACCIÓN
print(f"Descargando el artículo: '{PAGE_TITLE}'...")
wiki = wikipediaapi.Wikipedia(
    user_agent='MiProyectoRAG-Task2',
    language='en'
)
page = wiki.page(PAGE_TITLE)

if not page.exists():
    print(f"Error: La página '{PAGE_TITLE}' no existe.")
else:
    print("Página descargada con éxito.")

    # CHUNKING
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,     # Aprox. 300-350 palabras
        chunk_overlap=150,
        length_function=len
    )

    chunks = text_splitter.split_text(page.text)
    print(f"Texto dividido en {len(chunks)} chunks.")

    # GUARDADO
    df = pd.DataFrame({
        'id': [f"{PAGE_TITLE}_{i}" for i in range(len(chunks))],
        'title': [PAGE_TITLE] * len(chunks),
        'text': chunks
    })

    df.to_csv(CSV_PATH, index=False)
    print(f"¡Éxito! Corpus guardado en '{CSV_PATH}'")

    # Mostramos los primeros 5 para verificar
    print("\nVerificación del corpus (primeros 5 chunks):")
    display(df.head())

Creación del Dataset
Descargando el artículo: 'Federated_learning'...
Página descargada con éxito.
Texto dividido en 31 chunks.
¡Éxito! Corpus guardado en 'data/wiki_corpus.csv'

Verificación del corpus (primeros 5 chunks):


,id,title,text
0,Federated_learning_0,Federated_learning,Federated learning (also known as collaborativ...
1,Federated_learning_1,Federated_learning,Definition\nFederated learning aims at trainin...
2,Federated_learning_2,Federated_learning,The main difference between federated learning...
3,Federated_learning_3,Federated_learning,Mathematical formulation\nThe objective functi...
4,Federated_learning_4,Federated_learning,where \n \n \n \n K\n \n ...


In [3]:
# Creación de la Base de Datos Vectorial (Embedding)
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

print(f"Creación de la Base de Datos Vectorial")

# Cargar nuestro corpus
try:
    df = pd.read_csv(CSV_PATH)
    print(f"Corpus cargado desde '{CSV_PATH}' ({len(df)} chunks).")
except FileNotFoundError:
    print(f"Error: El archivo '{CSV_PATH}' no se encontró.")
    print("Por favor, vuelve a ejecutar la Celda 3 primero.")
    # Detener la ejecución si el archivo no existe
    raise

# Definir el modelo de embedding
print("Cargando el modelo de embeddings (Sentence Transformer)...")
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("Modelo de embeddings cargado.")

# Transformar los datos de DataFrame a lista de 'Documentos'

documents = [
    Document(
        page_content=row['text'],
        metadata={
            'id': row['id'],
            'title': row['title']
        }
    ) for index, row in df.iterrows()
]

# Crear y Guardar la Base de Datos Vectorial

print(f"Creando y guardando la base de datos vectorial en '{DB_DIR}'...")
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_function,
    persist_directory=DB_DIR
)
print("\n--- ¡Éxito! Base de datos vectorial creada y guardada. ---")
print(f"Total de vectores en la base: {db._collection.count()}")

Creación de la Base de Datos Vectorial
Corpus cargado desde 'data/wiki_corpus.csv' (31 chunks).
Cargando el modelo de embeddings (Sentence Transformer)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo de embeddings cargado.
Creando y guardando la base de datos vectorial en 'chroma_db_store'...


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



--- ¡Éxito! Base de datos vectorial creada y guardada. ---
Total de vectores en la base: 31


In [4]:
# Cargar el LLM Rápido

from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

# Definimos el nombre del modelo
MODEL_ID = "google/flan-t5-large"

print(f"Carga del LLM: {MODEL_ID}")

# Creamos el pipeline de la biblioteca transformers
llm_pipeline = pipeline(
    "text2text-generation",
    model=MODEL_ID,
    max_length=512,

    # --- ¡Clave para la Velocidad! ---
    # Asigna el pipeline al T4 GPU (dispositivo 'cuda:0')
    # Si no hay GPU, usa CPU (device=-1)
    device=0 if torch.cuda.is_available() else -1
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

print("\n--- ¡Éxito! LLM cargado y listo para usar. ---")

Carga del LLM: google/flan-t5-large


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



--- ¡Éxito! LLM cargado y listo para usar. ---


In [5]:
# Ensamblar el Pipeline de Q&A

from langchain.chains import RetrievalQA

print("Iniciando Paso 4: Ensamblaje del Pipeline de Q&A")

#  Crear el Retriever

retriever = db.as_retriever(
    search_kwargs={"k": 3}
)
print("Retriever (Buscador) creado.")

qa_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"

)
print("\n--- ¡Éxito! Pipeline de Q&A listo para usar. ---")

Iniciando Paso 4: Ensamblaje del Pipeline de Q&A
Retriever (Buscador) creado.

--- ¡Éxito! Pipeline de Q&A listo para usar. ---


In [6]:
# Generar el Resumen Principal (rag_summary.md)
import os

# Definimos una consulta amplia para el resumen
summary_query = (
    "Generate a comprehensive summary of federated learning, "
    "at least 400 words long. Explain what it is, "
    "its main characteristics, and its core concepts."
)

print(f"Ejecutando consulta para el resumen principal...")
print("El LLM (flan-t5) está generando la respuesta.")

# Ejecutamos el pipeline
summary_text = qa_pipeline.invoke(summary_query)["result"]

print("\n--- Resumen Generado ---")
print(summary_text)

# 3. Guardamos el resumen en el archivo
summary_file_path = os.path.join(OUTPUT_DIR, "rag_summary.md")
try:
    with open(summary_file_path, 'w', encoding='utf-8') as f:
        f.write(summary_text)
    print(f"\n¡Éxito! Resumen guardado en '{summary_file_path}'")

except Exception as e:
    print(f"\nError al guardar el resumen: {e}")

ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


Ejecutando consulta para el resumen principal...
El LLM (flan-t5) está generando la respuesta.

--- Resumen Generado ---
Federated learning is a machine learning technique in a setting where multiple entities (often called clients) collaboratively train a model while keeping their data decentralized, rather than centrally stored. A defining characteristic of federated learning is data heterogeneity. Because client data is decentralized, data samples held by each client may not be independently and identically distributed. Federated learning is generally concerned with and motivated by issues such as data privacy, data minimization, and data access rights. Its applications involve a variety of research areas including defence, telecommunications, the Internet of things, and pharmaceuticals.

¡Éxito! Resumen guardado en 'outputs/rag_summary.md'


In [7]:
# Generar Ejemplos de Q&A (retrieval_examples.json)

import json
import os

# Definimos las preguntas específicas

preguntas = [
    "Explain federated learning challenges in healthcare.",
    "What is the basic principle or definition of federated learning?"
]

print(f"Generando {len(preguntas)} ejemplos de Q&A...")

# Creamos una lista para guardar los resultados
resultados = []
# Iteramos sobre cada pregunta
for query in preguntas:
    print(f"\nEjecutando consulta: '{query}'")

    respuesta = qa_pipeline.invoke(query)["result"]

    resultados.append({
        "pregunta": query,
        "respuesta": respuesta
    })
    print("Respuesta generada.")

print("\n--- Todos los ejemplos generados ---")

# Guardamos la lista de resultados en un archivo JSON
json_file_path = os.path.join(OUTPUT_DIR, "retrieval_examples.json")
try:
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(resultados, f, indent=4, ensure_ascii=False)

    print(f"\n¡Éxito! Ejemplos guardados en '{json_file_path}'")

except Exception as e:
    print(f"\nError al guardar el archivo JSON: {e}")

ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Generando 2 ejemplos de Q&A...

Ejecutando consulta: 'Explain federated learning challenges in healthcare.'


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Respuesta generada.

Ejecutando consulta: 'What is the basic principle or definition of federated learning?'
Respuesta generada.

--- Todos los ejemplos generados ---

¡Éxito! Ejemplos guardados en 'outputs/retrieval_examples.json'
